To deliver personalized experiences to customers, segmentation is key. It can provide insights into your customers behavior, habits, and preferences, allowing you to offer tailored marketing campaigns increasing your odds of success as well as improving your customers experience with tailored content.

# RFM Model

Using transactional purchasing data, we will be able to create a 2 x 2 value matrix to create 4 customers groups. Each group will defer from the other depending on 2 dimensions: (1) current customer value, and (2) potential customer value.

<img src="https://miro.medium.com/max/1920/1*AW-LW6i7BPBs4ED3hCCd8w.png" width="400">

We are going to use the RFM model to create the required features from transactional purchasing data. The RFM model stands for:

- Recency: When was the last time they purchased?

- Frequency: How often and for how long have they purchased?

- Monetary Value/Sales: How much have they purchased?

It’s usually used to identify the Highest Value Customer at the intersection of each 3 questions. To build the 2 x 2 matrix we will only use the R & the M from RFM.

<img src="https://miro.medium.com/max/1088/1*7JfiT-GBi4R501svnVrQFA.png"  width="200">

**Step 0: Load, filter, clean, and aggregate the data at the customer level**

In [ ]:
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import datetime
path = "Put your data path here"
df = pd.read_excel(path)
df.drop(0,inplace = True)
df["Order Date"] = df["Payment No."].str[:8]
df = df[["Order Number","User ID","Order Date","Total GMV"]]

In [ ]:
df.head()

**Step 1: Create RFM Features for each customers**

In [ ]:
df_RFM = df.groupby('User ID').agg({'Order Date': lambda y: (df['Order Date'].max().date() - y.max().date()).days,
                                        'Order Number': lambda y: len(y.unique()),  
                                        'Total GMV': lambda y: round(y.sum(),2)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

**Step 2: To automate the segmentation we will use the 80% quantile for Recency and Monetary (we could have also used k-mean clustering or leveraged business knowledge to create buckets — for example global superstore business users consider an active customer as someone whose last order is less than 100 days old)**

In [ ]:
# We will use the 80% quantile for each feature
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM.head()

**Step 3: Calculate RM score and sort customers**

In [ ]:
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'User ID': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.head()

**Step 4: Visualize the Value Matrix and explore some key numbers**

In [ ]:
# 1) Average Monetary Matrix
df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_M

In [ ]:
# 2) Number of Customer Matrix
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='User ID')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_C

In [ ]:
# 3) Recency Matrix
df_RFM_R = df_RFM_SUM.pivot(index='M', columns='R', values='Recency')
df_RFM_R= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_R

<img src = "https://miro.medium.com/max/1920/1*zESqrQ1e6Tm5Uwu23PFtGA.png" width = "400">

**Some takeaways / quick wins with very simple sales & marketing tactics examples?**

- There are few customers in the “Disengaged” bucket and they have an average revenue lower than the “Star” bucket. Since there are very few of them, it should be easy to partner with the business to understand what happened at the customer level. Based on the analysis, there might be a simple quick win: reactivate few of them with a phone call or meeting to hopefully move them back to the “Star” bucket (e.g. engaged customers).
 
- The average last order from the “Light” bucket is very old (almost 1 year vs. 60-70 days for ‘engaged’ customers). Launching a simple reactivation campaign with a coupon might be an initiative that could lead to some new orders and help some of these customers move to the “New” bucket (e.g. engaged customers).

<img src = "https://miro.medium.com/max/1080/1*wZY7pxdnVgv-34v7gRQ9jw.png" width = "300">